# Continuación webcrawling/scrapping para Seo

# Ejemplo 1: Extraer el título y la meta descripción
- Aplicación SEO: Verificamos que cada página tenga un título adecuado y una descripción optimizada.



In [75]:
import requests
from bs4 import BeautifulSoup

url = "https://www.python.org"
url2 = "https://webscraper.io/test-sites/e-commerce/static"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Título de la página
titulo = soup.title.string

# Meta descripción
meta_descripcion = soup.find('meta', attrs={'name': 'description'})
# descripcion = meta_descripcion['content'] if meta_descripcion else "No tiene descripción"

if meta_descripcion:
    descripcion = meta_descripcion['content']
else:
    descripcion = "No tiene descripción"

print("Título:", titulo)
print("Descripción:", descripcion)
#print(soup.prettify())


Título: Welcome to Python.org
Descripción: The official home of the Python Programming Language


# Ejemplo 2: Revisar estructura de encabezados H1, H2, H3

Aplicación SEO: Un solo <h1>, uso jerárquico de encabezados. Puedes revisar si una página está correctamente estructurada.

In [46]:
encabezados = {}
for tag in ['h1', 'h2', 'h3']:
    encabezados[tag] = [h.get_text(strip=True) for h in soup.find_all(tag)]

print(encabezados)


{'h1': ['', 'Functions Defined', 'Compound Data Types', 'Intuitive Interpretation', 'All the Flow You’d Expect', 'Quick & Easy to Learn'], 'h2': ['Get Started', 'Download', 'Docs', 'Jobs', 'Latest News', 'Upcoming Events', 'Success Stories', 'Use Python for…', '>>>Python Software Foundation'], 'h3': []}


# Ejemplo 3: Verificar si hay enlaces rotos (broken links)
Aplicación SEO: Detectar enlaces rotos que afectan el posicionamiento y la experiencia del usuario.

In [47]:
from urllib.parse import urljoin

enlaces = soup.find_all('a', href=True)
for enlace in enlaces[:10]:  # Limita para pruebas
    link = urljoin(url2, enlace['href'])
    try:
        r = requests.head(link, timeout=5)
        if r.status_code >= 400:
            print("Enlace roto:", link, "Status:", r.status_code)
        else:
            print("Enlace OK:", r.status_code)
    except requests.RequestException:
        print("Enlace roto:", link)


Enlace OK: 200
Enlace OK: 200
Enlace OK: 200
Enlace OK: 301
Enlace OK: 302
Enlace OK: 200
Enlace OK: 200
Enlace roto: https://webscraper.io/community/ Status: 404
Enlace OK: 200
Enlace OK: 200


# Ejemplo 4: Verificar imágenes sin atributo alt
Aplicación SEO: Las imágenes sin atributo alt perjudican la accesibilidad y el SEO de imágenes.



In [38]:
imagenes = soup.find_all('img')
sin_alt = [img['src'] for img in imagenes if not img.get('alt')]

print("Imágenes sin 'alt':", sin_alt)


Imágenes sin 'alt': []


# Ejemplo 5: Verificar la presencia del atributo canonical
El atributo rel="canonical" evita contenido duplicado indicando la versión principal de una página.

Aplicación SEO: Asegura que se indica cuál es la URL original de la página.



In [55]:
canonical = soup.find('link', rel='canonical')
if canonical:
    print("Canonical:", canonical['href'])
else:
    print("No se encontró etiqueta canonical.")


Canonical: https://webscraper.io/test-sites/e-commerce/static


# Ejemplo 6: Listar todas las URLs internas y externas
Aplicación SEO: Ayuda a entender la estructura de enlaces internos y salientes.



In [56]:
from urllib.parse import urlparse

internas = []
externas = []

for a in soup.find_all('a', href=True):
    href = a['href']
    if href.startswith('#') or href.startswith('mailto:') or href.startswith('tel:'):
        continue
    if urlparse(href).netloc == "" or urlparse(href).netloc == urlparse(url).netloc:
        internas.append(href)
    else:
        externas.append(href)

print("Enlaces internos:", internas[:5])
print("Enlaces externos:", externas[:5])


Enlaces internos: ['/', '/', '/cloud-scraper', '/pricing', '/documentation']
Enlaces externos: ['https://forum.webscraper.io/', 'https://chromewebstore.google.com/detail/web-scraper-free-web-scra/jnhgnonknehpejjnehehllkliplmbmhn?hl=en', 'https://cloud.webscraper.io/', 'https://webscraper.io/downloads/Web_Scraper_Media_Kit.zip', 'https://forum.webscraper.io/']


# Ejemplo 7: Revisar si el archivo robots.txt permite el acceso
Aplicación SEO: Verifica si la página permite ser rastreada por motores de búsqueda.



In [59]:
robots_url = url + "/robots.txt"
try:
    r = requests.get(robots_url)
    print("Contenido de robots.txt:\n", r.text[:300])
except:
    print("No se pudo obtener robots.txt")


Contenido de robots.txt:
 # Directions for robots.  See this URL:
# http://www.robotstxt.org/robotstxt.html
# for a description of the file format.

User-agent: HTTrack
User-agent: puf
User-agent: MSIECrawler
Disallow: /

# The Krugle web crawler (though based on Nutch) is OK.
User-agent: Krugle
Allow: /
Disallow: /~guido/or


# Ejemplo 8: Detectar páginas sin etiquetas <meta name="robots">
Aplicación SEO: Se puede indicar noindex, nofollow, etc. Esta etiqueta debe estar bien configurada.



In [62]:
robots_meta = soup.find('meta', attrs={'name': 'robots'})
if robots_meta:
    print("Meta robots:", robots_meta['content'])
else:
    print("No se encontró etiqueta meta robots.")


No se encontró etiqueta meta robots.


# Ejemplo 9: Verificar si está el sitemap.xml

In [68]:
import requests



# 2. Posibles rutas de sitemap
posibles_sitemaps = [url, url2]

# 3. Verificación de existencia
for path in posibles_sitemaps:

    try:
        response = requests.head(path, timeout=5)
        if response.status_code == 200:
            print(f"✅ Sitemap encontrado en: {path}")
        else:
            print(f"❌ No encontrado: {path} (Status {response.status_code})")
    except Exception as e:
        print(f"⚠️ Error al acceder a {path}: {e}")



✅ Sitemap encontrado en: https://www.python.org
✅ Sitemap encontrado en: https://webscraper.io/test-sites/e-commerce/static


# Ejemplo 10: Detectar múltiples etiquetas <title> (error común)
Aplicación SEO: Tener más de un <title> puede confundir a los motores de búsqueda.



In [71]:
titulos = soup.find_all('title')
if len(titulos) > 1:
    print(f"¡Cuidado! Hay {len(titulos)} etiquetas <title>")
else:
    print("Etiqueta <title> correcta:", titulos[0].string if titulos else "No hay título")


Etiqueta <title> correcta: Static | Web Scraper Test Sites


# Ejemplo 11: Detectar páginas sin favicon
Aplicación SEO: Aunque no afecta directamente al ranking, es importante para la apariencia en resultados y marcadores.

In [73]:
favicon = None
for link in soup.find_all("link"):
    rel = link.get("rel")
    if rel and any("icon" in r.lower() for r in rel):
        favicon = link
        break

if favicon and favicon.get("href"):
    print("Favicon encontrado:", favicon["href"])
else:
    print("No tiene favicon")


Favicon encontrado: /favicon.png


# Ejemplo 12: Verificar el uso de etiquetas <strong> y <em>
Aplicación SEO: Útil para destacar contenido relevante y palabras clave.



In [ ]:
strongs = soup.find_all('strong')
ems = soup.find_all('em')

print(f"<strong>: {len(strongs)} - <em>: {len(ems)}")

if len(strongs) + len(ems) == 0:
    print("No se están destacando palabras clave con <strong> o <em>")

# Ejemplo 13: EVerificar uso de Open Graph (Facebook) y Twitter Cards


In [80]:
og_tags = soup.find_all('meta', property=lambda p: p and p.startswith('og:'))
twitter_tags = soup.find_all('meta', attrs={'name': lambda n: n and n.startswith('twitter:')})

print("Etiquetas Open Graph:", [tag['property'] for tag in og_tags])
print("Etiquetas Twitter:", [tag['name'] for tag in twitter_tags])


Etiquetas Open Graph: ['og:type', 'og:site_name', 'og:title', 'og:description', 'og:image', 'og:image:secure_url', 'og:url']
Etiquetas Twitter: []


# Ejemplo 14: Extraer y analizar la densidad de palabras clave

In [83]:
import re
from collections import Counter

texto = soup.get_text().lower()
palabras = re.findall(r'\b\w{4,}\b', texto)  # solo palabras de 4 letras o más
stopwords = {'de', 'la', 'y', 'el', 'en', 'los', 'las', 'con', 'por', 'una', 'para', 'que', 'del', 'sus', 'más'}

# Filtra y cuenta
palabras_filtradas = [p for p in palabras if p not in stopwords]
top_keywords = Counter(palabras_filtradas).most_common(10)

print("Top 10 palabras clave:", top_keywords)


Top 10 palabras clave: [('python', 56), ('2025', 15), ('news', 12), ('events', 11), ('more', 11), ('community', 9), ('with', 8), ('docs', 7), ('code', 6), ('other', 6)]


# EJERCICIO PRÁCTICO

In [86]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse

def analyze_url(url):
    result = {
        "URL": url,
        "Title": None,
        "Title Length": None,
        "Meta Description": None,
        "Meta Description Length": None,
        "H1 Count": None,
        "Images Missing Alt": None,
        "Canonical Tag": None,
        "Internal Links Count": None,
        "External Links Count": None,
        "Broken Links Count": None,
        "Broken Links Sample": None,
    }

    try:
        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            result["Title"] = f"Error: status {response.status_code}"
            return result

        soup = BeautifulSoup(response.text, 'html.parser')

        # Título
        if soup.title and soup.title.string:
            title = soup.title.string.strip()
            result["Title"] = title
            result["Title Length"] = len(title)
        else:
            result["Title"] = "No Title"
            result["Title Length"] = 0

        # Meta descripción
        meta_desc = soup.find("meta", attrs={"name": "description"})
        if meta_desc and meta_desc.get("content"):
            desc = meta_desc["content"].strip()
            result["Meta Description"] = desc
            result["Meta Description Length"] = len(desc)
        else:
            result["Meta Description"] = "No Meta Description"
            result["Meta Description Length"] = 0

        # H1
        h1_tags = soup.find_all("h1")
        result["H1 Count"] = len(h1_tags)

        # Imágenes sin alt
        images = soup.find_all("img")
        missing_alt = sum(1 for img in images if not img.get("alt"))
        result["Images Missing Alt"] = missing_alt

        # Canonical
        canonical = soup.find("link", rel=lambda val: val and "canonical" in val.lower())
        result["Canonical Tag"] = canonical["href"] if canonical and canonical.get("href") else "No canonical"

        # Enlaces
        parsed_base = urlparse(url)
        internal_links = []
        external_links = []
        broken_links = []

        for a in soup.find_all("a", href=True):
            href = a["href"]
            if href.startswith("#") or href.startswith("mailto:") or href.startswith("tel:"):
                continue
            full_link = urljoin(url, href)
            parsed_link = urlparse(full_link)
            if parsed_link.netloc == "" or parsed_link.netloc == parsed_base.netloc:
                internal_links.append(full_link)
            else:
                external_links.append(full_link)

            # Comprobación de enlace roto
            try:
                r = requests.head(full_link, allow_redirects=True, timeout=5)
                if r.status_code >= 400:
                    broken_links.append(full_link)
            except:
                broken_links.append(full_link)

        result["Internal Links Count"] = len(internal_links)
        result["External Links Count"] = len(external_links)
        result["Broken Links Count"] = len(broken_links)
        result["Broken Links Sample"] = "\n".join(broken_links[:3]) if broken_links else "Ninguno"

    except Exception as e:
        result["Title"] = f"Error: {e}"

    return result

def main():
    urls = [
        # "http://books.toscrape.com/",
        # "http://quotes.toscrape.com/",
        # "https://demo.opencart.com/"
        "https://www.python.org",
        "https://webscraper.io/test-sites/e-commerce/static"
    ]

    results = []
    for url in urls:
        print(f"Analizando: {url}")
        data = analyze_url(url)
        results.append(data)

    df = pd.DataFrame(results)
    df.to_excel("seo_audit_with_broken_links.xlsx", index=False)
    print("¡Informe completo exportado a seo_audit_with_broken_links.xlsx!")

if __name__ == "__main__":
    main()


Analizando: https://www.python.org
Analizando: https://webscraper.io/test-sites/e-commerce/static
¡Informe completo exportado a seo_audit_with_broken_links.xlsx!


# Ejercicio: Analizar palabras clave SEO y exportar a Excel
## Objetivo

- Extraer el texto visible de una página web.

- Contar cuántas veces aparece cada palabra.

- Eliminar las stopwords (palabras vacías comunes).

```

stopwords = ['de', 'la', ...']'
```


- Quedarse con las más relevantes (longitud > 3).

- Exportar a un archivo Excel con las columnas:
Palabra, Frecuencia.



---



✅ **Test de Autoevaluación: SEO On-Page con Python (20 preguntas)**
**Instrucciones:**  Elige la opción correcta en cada pregunta. Solo una respuesta es válida por ítem.


---


🧠 **1. Fundamentos y herramientas**

**1.1.**  ¿Qué librería en Python es más adecuada para analizar el HTML de una página?

A) urllib

B) re

C) BeautifulSoup

D) time


**1.2.**  ¿Cuál de estas librerías se utiliza para hacer peticiones HTTP?

A) matplotlib

B) requests

C) pandas

D) os

**1.3.**  ¿Qué librería se usa para exportar datos a Excel desde Python?

A) numpy

B) selenium

C) pandas

D) hashlib

**1.4.**  ¿Cuál es el objetivo principal de BeautifulSoup?

A) Controlar formularios

B) Parsear (analizar) HTML y XML

C) Dibujar gráficos

D) Descargar archivos binarios

**1.5.**  ¿Qué función tiene `urljoin()` en el análisis de enlaces?

A) Analiza direcciones IP

B) Convierte URL a mayúsculas

C) Une URLs base con relativas

D) Divide una URL en partes


---


🔍 **2. Elementos SEO on-page**

**2.1.**  ¿Cuál es la etiqueta principal que debe representar el título del contenido?

A) `<title>`

B) `<h1>`

C) `<meta>`

D) `<header>`

**2.2.**  ¿Cuál es la longitud recomendada para una etiqueta `<title>`?

A) Máximo 20 caracteres

B) Entre 60 y 70 caracteres

C) Más de 100 caracteres

D) No tiene límite

**2.3.**  ¿Qué atributo de la imagen se analiza para evaluar la accesibilidad y el SEO?

A) `src`

B) `href`

C) `alt`

D) `class`

**2.4.**  ¿Cuál de estos códigos HTTP indica un enlace roto?

A) 200

B) 301

C) 403

D) 404

**2.5.**  ¿Qué metaetiqueta se usa para evitar que una página sea indexada?

A) `<meta name="robots" content="noindex">`

B) `<meta charset="UTF-8">`

C) `<meta name="canonical">`

D) `<meta name="nofollow">`


---


✍️ **3. Análisis de contenido y palabras clave**

**3.1.**  ¿Qué hace la función `get_text()` en BeautifulSoup?

A) Elimina los enlaces

B) Extrae todo el texto visible del HTML

C) Extrae solo el contenido de imágenes

D) Corta el contenido

**3.2.**  ¿Qué son las “stopwords”?

A) Enlaces rotos

B) Palabras que deben destacarse

C) Palabras comunes que se excluyen del análisis

D) Palabras duplicadas

**3.3.**  ¿Para qué se usa `collections.Counter()`?

A) Ordenar listas alfabéticamente

B) Contar elementos repetidos

C) Dividir cadenas de texto

D) Verificar enlaces rotos

**3.4.**  ¿Qué estructura de datos devuelve `Counter(palabras)`?

A) Lista

B) Diccionario

C) Tupla

D) Conjunto (set)

**3.5.**  ¿Cuál es un buen indicador de que una página tiene contenido suficiente para SEO?

A) Más de 50 palabras

B) Más de 100 enlaces

C) Más de 300 palabras de contenido real

D) Tener un sitemap


---


🧪 **4. Práctica con enlaces, imágenes y exportación**

**4.1.**  ¿Cómo detectamos imágenes sin atributo `alt` con BeautifulSoup?

A) `img.get("src") == None`

B) `img.get("alt") == None`

C) `img.has_class("alt")`

D) `img.text == ""`


**4.2.**  ¿Qué atributo usamos para identificar un enlace canónico?

A) `rel="canonical"`

B) `name="robots"`

C) `meta="description"`

D) `href="canonical"`


**4.3.**  ¿Qué diferencia hay entre un enlace interno y uno externo?

A) El interno usa `mailto:`

B) El externo tiene un dominio distinto

C) El externo no tiene protocolo

D) El interno es siempre un script


**4.4.**  ¿Qué código HTTP representa acceso prohibido?

A) 302

B) 200

C) 403

D) 100

**4.5.**  ¿Qué función usamos en pandas para guardar un DataFrame en Excel?

A) `save_as_excel()`

B) `export_excel()`

C) `to_excel()`

D) `write_xls()`


---


🧩 **5. Verificación de estructura SEO**

**5.1.**  ¿Qué pasa si una página tiene varios `<h1>`?

A) Mejora el SEO

B) Se considera contenido duplicado

C) Rompe el HTML

D) Afecta la jerarquía semántica

**5.2.**  ¿Por qué es importante verificar enlaces rotos?

A) Rompen la maquetación

B) Penalizan el SEO y afectan al usuario

C) Solo son molestos en móviles

D) Impiden cargar CSS

**5.3.**  ¿Qué extensión debe tener el archivo de un sitemap estándar?

A) `.html`

B) `.txt`

C) `.json`

D) `.xml`

**5.4.**  ¿Qué significa que una página tenga un “canonical tag”?

A) Es la página más vista

B) Es la versión original para evitar duplicación

C) Es la que carga más rápido

D) Es la última página actualizada

**5.5.**  ¿Dónde se suele declarar el `favicon` en HTML?

A) En `<footer>`

B) En `<body>`

C) En `<link rel="icon">` dentro de `<head>`

D) En `robots.txt`